In [1]:
import os

file_path = os.getcwd()
print(file_path)

# file_path= '/Users/ckana/Downloads/header'  # html documents local directory

html_files = list(filter(lambda x: '.html' in x, os.listdir(file_path)))
html_files

In [2]:
local_path = 'file:///'+file_path+'/'
# local_path = 'file:///Users/ckana/Downloads/header/'  # absolute path for html documents
new_html_path = [local_path + x for x in html_files]
new_html_path

In [3]:
import scrapy
from scrapy.crawler import CrawlerProcess
import unicodedata

header = {'Case':[],'Status':[],'Judge':[]}


class PacerSpider(scrapy.Spider):
    name = 'pacer_header_spider'
    
    def start_requests(self):
        urls = new_html_path # absolute path
        for url in urls:
            yield scrapy.Request(url = url, callback = self.parse, dont_filter=True)
    
    def parse(self, response):
        case_title = response.css('div#cmecfMainContent > center:nth-of-type(1) ::text').extract()
        for i in range(len(case_title)):
            case_title[i] = case_title[i].replace('\r\n',' ')        
        # print(case_title)
        header['Case'].append(case_title[2])

        
        case_status = response.css('div#cmecfMainContent').xpath('table[1]//h2//text()').extract()
        for i in range(len(case_status)):
            case_status[i] = case_status[i].replace('\r\n',' ')
            case_status[i] = unicodedata.normalize("NFKD",case_status[i]).strip()
        
        if '' in case_status:
            case_status.remove('')
        # print(case_status)
        header['Status'].append(case_status[-1])
        
        
        judge_name = response.css('div#cmecfMainContent').xpath('table[2]/tr/td[1]//text()').extract()
        for i in range(len(judge_name)):
            judge_name[i] = judge_name[i].replace('\r\n',' ')
            judge_name[i] = unicodedata.normalize("NFKD",judge_name[i]).strip()
        
        if '' in judge_name:
            judge_name.remove('')
        # print(judge_name)
        header['Judge'].append(judge_name[1])

        
        dates = response.css('div#cmecfMainContent').xpath('table[2]/tr/td[2]//text()').extract()
        # print(dates)
        for i in range(len(dates)):
            dates[i] = unicodedata.normalize("NFKD",dates[i]).strip()

        if '' in dates:
            dates.remove('')
        # print(dates)
        
        
        for i in range(0,len(dates),2):
            if dates[i] not in header: # new date column
                header[dates[i]] = [''] * (len(header['Case'])-1)
                header[dates[i]].append(dates[i+1])
            else:
                header[dates[i]].append(dates[i+1])
        
        for key in header:
            if key not in dates and key not in ['Case','Status','Judge']:
                header[key].append('')
        
        # print(header)
      
       

process = CrawlerProcess(settings={
    "FEEDS": {
        "items.json": {"format": "json"},
    },
})

process.crawl(PacerSpider)
process.start()

2020-08-02 11:54:55 [scrapy.utils.log] INFO: Scrapy 2.1.0 started (bot: scrapybot)
2020-08-02 11:54:55 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.7, Platform Windows-10-10.0.18362-SP0
2020-08-02 11:54:55 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-08-02 11:54:55 [scrapy.crawler] INFO: Overridden settings:
{}
2020-08-02 11:54:55 [scrapy.extensions.telnet] INFO: Telnet Password: 91386b013e2b56b7
2020-08-02 11:54:55 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2020-08-02 11:54:55 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermi

In [4]:
import pandas as pd

header_df = pd.DataFrame.from_dict(header)
header_df

2020-08-02 11:54:58 [matplotlib] DEBUG: $HOME=C:\Users\ckana
2020-08-02 11:54:58 [matplotlib] DEBUG: CONFIGDIR=C:\Users\ckana\.matplotlib
2020-08-02 11:54:58 [matplotlib] DEBUG: matplotlib data path: C:\Users\ckana\Anaconda3\lib\site-packages\matplotlib\mpl-data
2020-08-02 11:54:58 [matplotlib] DEBUG: loaded rc file C:\Users\ckana\Anaconda3\lib\site-packages\matplotlib\mpl-data\matplotlibrc
2020-08-02 11:54:58 [matplotlib] DEBUG: matplotlib version 3.1.1
2020-08-02 11:54:58 [matplotlib] DEBUG: interactive is False
2020-08-02 11:54:58 [matplotlib] DEBUG: platform is win32
2020-08-02 11:54:58 [matplotlib] DEBUG: loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', 'zipimport', '_frozen_importlib_external', '_io', 'marshal', 'nt', 'winreg', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_abc', '_bootlocale', '_locale', 'encodings.cp1252', 'site', 'os', 'stat

2020-08-02 11:54:59 [matplotlib] DEBUG: CACHEDIR=C:\Users\ckana\.matplotlib
2020-08-02 11:54:59 [matplotlib.font_manager] DEBUG: Using fontManager instance from C:\Users\ckana\.matplotlib\fontlist-v310.json


,Case,Status,Judge,Date filed:,Date terminated:,Plan confirmed:,Date converted:,341 meeting:,Debtor dismissed:
0,Bankruptcy Petition #: 07-10432-CSS,"LeadSC, CLOSED",Christopher S. Sontchi,04/02/2007,06/30/2008,04/17/2008,,,
1,Bankruptcy Petition #: 07-11018-KG,"CONVERTED, LeadSC, CLAIMS, ASSET, CLOSED",Kevin Gross,07/31/2007,09/17/2013,,06/11/2008,07/21/2008,
2,Bankruptcy Petition #: 99-00615-KJC,"LEAD, MEGA, CONVERTED, FUNDS, CLOSED",Kevin J. Carey,03/17/1999,01/10/2018,,02/21/2003,05/07/2003,
3,Bankruptcy Petition #: 08-10322-KG,"SealedDoc(s), MEGA, DISMISSED, CLOSED",Kevin Gross,02/19/2008,01/11/2013,,,,12/27/2012
4,Bankruptcy Petition #: 08-12027-PJW,"TRANSOUT, LEAD, MEGA, CLOSED",Peter J. Walsh,08/29/2008,07/01/2010,,,,
5,Bankruptcy Petition #: 09-11696-KJC,"CLAIMS, LEAD, CONVERTED, FUNDS, CLOSED",Kevin J. Carey,05/13/2009,03/04/2019,,12/01/2009,03/05/2010,
6,Bankruptcy Petition #: 10-13189-KJC,"CLOSED, LeadSC, CLAIMS, CONFIRMED",Kevin J. Carey,09/30/2010,07/01/2011,05/09/2011,,11/04/2010,
7,Bankruptcy Petition #: 12-13319-KG,"LEAD, MEGA, CLMSAGNT, CLOSED",Kevin Gross,12/11/2012,10/28/2016,,,03/12/2013,
8,Bankruptcy Petition #: 14-10367-PJW,"CLOSED, LEAD, MEGA, DISMISSED",Peter J. Walsh,02/24/2014,11/07/2014,,,03/26/2014,10/30/2014
9,Bankruptcy Petition #: 16-10569-KG,"DISMISSED, CLOSED",Kevin Gross,03/04/2016,05/24/2016,,,,05/09/2016


In [5]:
header_df.to_csv('PACER_header.csv',index=False)